<a href="https://colab.research.google.com/github/sergiocostaifes/PPCOMP_DM/blob/main/notebooks/01_ingest_validate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bootstrap padrão

In [4]:

# --- Bootstrap padrão PPCOMP_DM (Colab) ---
from google.colab import drive
drive.mount("/content/drive", force_remount=False)

import sys, subprocess, importlib
from pathlib import Path
from importlib.machinery import PathFinder

REPO_DIR = Path("/content/drive/MyDrive/Mestrado/PPCOMP_DM")
GITHUB_REPO = "https://github.com/sergiocostaifes/PPCOMP_DM.git"

# 1) Garantir repo (clone se faltar)
if not REPO_DIR.exists():
    REPO_DIR.parent.mkdir(parents=True, exist_ok=True)
    subprocess.run(["git", "clone", GITHUB_REPO, str(REPO_DIR)], check=True)

# 2) sys.path com prioridade
repo_str = str(REPO_DIR)
if repo_str in sys.path:
    sys.path.remove(repo_str)
sys.path.insert(0, repo_str)

# 3) Ajuste do Colab: garantir import normal de pacotes locais
importlib.invalidate_caches()
if PathFinder not in sys.meta_path:
    sys.meta_path.append(PathFinder)

# 4) Garantir src como pacote
(REPO_DIR / "src").mkdir(parents=True, exist_ok=True)
init_file = REPO_DIR / "src" / "__init__.py"
if not init_file.exists():
    init_file.write_text("# src package\n", encoding="utf-8")

print("REPO_DIR:", REPO_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
REPO_DIR: /content/drive/MyDrive/Mestrado/PPCOMP_DM


# Código ingest e validate

In [5]:
# =========================
# 01_ingest_validate.ipynb
# Ingestão + Sanidade
# =========================

# (crítico) garante que o Python não use módulo antigo em cache
importlib.invalidate_caches()
import os

# ===== Imports do projeto =====
from importlib import reload
import src.paths as _paths
reload(_paths)  # garante pegar a versão atual do arquivo

from src.paths import RAW_PATH, PROCESSED_PATH, FEATURES_PATH, MODELS_PATH, REPORTS_PATH, ensure_dirs
ensure_dirs()

print("Paths carregados:")
print("RAW_PATH:", RAW_PATH)
print("PROCESSED_PATH:", PROCESSED_PATH)

def log(msg: str) -> None:
    print(f"[01_ingest_validate] {msg}")

# ===== Libs =====
import pandas as pd
import numpy as np

# 0) Arquivo de entrada
CSV_FILE = RAW_PATH / "borg_traces_data.csv"

if not CSV_FILE.exists():
    # diagnóstico detalhado (não fica “cego”)
    log(f"Arquivo não encontrado: {CSV_FILE}")
    log(f"Listando RAW_PATH: {RAW_PATH}")
    try:
        log(str(os.listdir(RAW_PATH)))
    except Exception as e:
        log(f"Falha ao listar RAW_PATH: {e}")
    raise FileNotFoundError(f"Arquivo não encontrado: {CSV_FILE}")

log(f"Arquivo encontrado: {CSV_FILE}")

# 1) Leitura (opções melhores para Colab)
df = pd.read_csv(
    CSV_FILE,
    low_memory=False
)
log(f"Leitura OK: {df.shape} (linhas, colunas)")
log(f"Colunas: {list(df.columns)}")

# 2) Remover coluna lixo se existir
df = df.drop(columns=["Unnamed: 0"], errors="ignore")

# 3) time: numérico + limpeza de sentinela (regra Dia 1)
if "time" not in df.columns:
    raise KeyError("Coluna 'time' não encontrada no dataset.")

df["time"] = pd.to_numeric(df["time"], errors="coerce")
sentinel_cutoff = 1e16
n_before = int(df["time"].isna().sum())
df.loc[df["time"] > sentinel_cutoff, "time"] = np.nan
n_after = int(df["time"].isna().sum())

log(f"time NaNs antes: {n_before} | depois de sentinela: {n_after}")
log(f"time min: {df['time'].min()}")
log(f"time median: {df['time'].median()}")
log(f"time max: {df['time'].max()}")
if df["time"].min() == 0:
    log("Observação: time==0 será tratado no 02_clean_normalize (remoção/ajuste conforme regra do projeto).")

# 5) Validar falha: FAIL vs failed
if "event" in df.columns and "failed" in df.columns:
    concord = (df["event"].eq("FAIL") == df["failed"].eq(1)).mean()
    log(f"Concordância event=='FAIL' vs failed==1: {concord:.4f}")
else:
    log("Colunas event/failed não encontradas; pulando checagem de concordância.")

# 6) Relatório curto (visão RAW - antes da seleção KEEP_COLS)
summary = {
    "raw_rows": int(df.shape[0]),
    "raw_cols": int(df.shape[1]),
    "raw_time_nan": int(df["time"].isna().sum()),
    "raw_failed_1_count": int((df["failed"] == 1).sum()) if "failed" in df.columns else None,
    "raw_event_top": df["event"].value_counts().head(10).to_dict() if "event" in df.columns else None,
}
summary

Paths carregados:
RAW_PATH: /content/drive/MyDrive/Mestrado/02-datasets/01-raw
PROCESSED_PATH: /content/drive/MyDrive/Mestrado/02-datasets/02-processed
[01_ingest_validate] Arquivo encontrado: /content/drive/MyDrive/Mestrado/02-datasets/01-raw/borg_traces_data.csv
[01_ingest_validate] Leitura OK: (405894, 34) (linhas, colunas)
[01_ingest_validate] Colunas: ['Unnamed: 0', 'time', 'instance_events_type', 'collection_id', 'scheduling_class', 'collection_type', 'priority', 'alloc_collection_id', 'instance_index', 'machine_id', 'resource_request', 'constraint', 'collections_events_type', 'user', 'collection_name', 'collection_logical_name', 'start_after_collection_ids', 'vertical_scaling', 'scheduler', 'start_time', 'end_time', 'average_usage', 'maximum_usage', 'random_sample_usage', 'assigned_memory', 'page_cache_memory', 'cycles_per_instruction', 'memory_accesses_per_instruction', 'sample_rate', 'cpu_usage_distribution', 'tail_cpu_usage_distribution', 'cluster', 'event', 'failed']
[01_ing

{'raw_rows': 405894,
 'raw_cols': 33,
 'raw_time_nan': 3,
 'raw_failed_1_count': 92678,
 'raw_event_top': {'FINISH': 92867,
  'FAIL': 92678,
  'ENABLE': 75907,
  'SCHEDULE': 69104,
  'LOST': 59515,
  'EVICT': 14756,
  'KILL': 951,
  'UPDATE_PENDING': 111,
  'QUEUE': 4,
  'UPDATE_RUNNING': 1}}

# Persistência do dataset validado

In [6]:
# =========================
# Persistência do dataset validado (Parquet)
# =========================

# Confirmar que o pyarrow está instalado
import importlib.util, subprocess, sys
if importlib.util.find_spec("pyarrow") is None:
    subprocess.run([sys.executable, "-m", "pip", "-q", "install", "pyarrow"], check=True)

# KEEP_COLS = [ # SIMPLE
#     "time",
#     "failed",
#     "event",
#     "machine_id",
#     "collection_id",
#     "instance_index",
#     "priority",
#     "scheduling_class",
#     "start_time",
#     "end_time",
# ]

KEEP_COLS = [  # EXTENDED
    "time","failed","event","machine_id","collection_id","instance_index",
    "priority","scheduling_class","start_time","end_time",
    "resource_request","average_usage","maximum_usage","random_sample_usage",
    "assigned_memory","page_cache_memory",
    "cycles_per_instruction","memory_accesses_per_instruction",
    "sample_rate","cluster","scheduler"
]

# Tipos recomendados (aplicados depois da leitura)
CAT_COLS = [c for c in ["event","cluster","scheduler","scheduling_class","priority"] if c in KEEP_COLS]
FLOAT32_COLS = [c for c in [
    "average_usage","maximum_usage","random_sample_usage",
    "assigned_memory","page_cache_memory",
    "cycles_per_instruction","memory_accesses_per_instruction",
    "sample_rate"
] if c in KEEP_COLS]

INT_COLS = [c for c in ["machine_id","collection_id","instance_index","start_time","end_time"] if c in KEEP_COLS]

# Releitura do CSV só com as colunas necessárias (mais rápido e leve)
df = pd.read_csv(CSV_FILE, usecols=KEEP_COLS, low_memory=False)

# Limpeza time + sentinela
df["time"] = pd.to_numeric(df["time"], errors="coerce")
sentinel_cutoff = 1e16
df.loc[df["time"] > sentinel_cutoff, "time"] = np.nan

# failed
df["failed"] = pd.to_numeric(df["failed"], errors="coerce").fillna(0).astype("int8")

# Categóricas
for c in CAT_COLS:
    df[c] = df[c].astype("category")

# Inteiros (nullable, para não estourar se houver NaN)
for c in INT_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

# Float32 (reduz memória)
for c in FLOAT32_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce").astype("float32")

# Remover linhas sem time (recomendado para o pipeline)
n_before = len(df)
df = df.dropna(subset=["time"]).copy()
df["time"] = df["time"].astype("int64")
print(f"Drop time NaN: {n_before} -> {len(df)}")

# =========================
# Salvar Parquet validado
# =========================
VALIDATED_FILE = PROCESSED_PATH / "trace_raw_validated.parquet"
df.to_parquet(VALIDATED_FILE, engine="pyarrow", compression="snappy", index=False)

log(f"Salvo: {VALIDATED_FILE}")
log(f"Tamanho (MB): {round(VALIDATED_FILE.stat().st_size/1024/1024, 2)}")

# =========================
# Atualizar summary final
# =========================
summary["parquet_rows"] = int(len(df))
summary["parquet_cols"] = int(df.shape[1])
summary["kept_cols"] = list(df.columns)
summary["validated_file"] = str(VALIDATED_FILE)

# =========================
# Persistir summary
# =========================
import json
summary_file = REPORTS_PATH / "01_ingest_validate_summary.json"
summary_file.write_text(
    json.dumps(summary, indent=2, ensure_ascii=False),
    encoding="utf-8"
)

log(f"Resumo salvo: {summary_file}")

Drop time NaN: 405894 -> 405891
[01_ingest_validate] Salvo: /content/drive/MyDrive/Mestrado/02-datasets/02-processed/trace_raw_validated.parquet
[01_ingest_validate] Tamanho (MB): 12.3
[01_ingest_validate] Resumo salvo: /content/drive/MyDrive/Mestrado/04-reports/01_ingest_validate_summary.json


In [ ]:
# Achados do Notebook 01 — Ingestão e Validação Inicial

## Leitura e estrutura do dataset bruto
- Arquivo de entrada: `borg_traces_data.csv`
- Total de registros lidos: **405.894**
- Total de colunas após remoção de coluna lixo (`Unnamed: 0`): **33**

Após remoção de `time` inválido (NaN decorrente de sentinela):
- Registros persistidos em parquet validado: **405.891**

## Sanidade temporal
- `time` convertido para numérico (int64)
- Valores acima do sentinela (`1e16`) tratados como inválidos
- `time == 0` identificado como caso especial (tratado formalmente no Notebook 02)

## Consistência semântica de falhas
- Concordância entre `event == "FAIL"` e `failed == 1`: **100%**
- Isso valida a consistência lógica entre a variável categórica de evento e a variável binária de falha.

## Distribuição dos principais eventos
Top eventos observados:
- FINISH
- FAIL
- ENABLE
- SCHEDULE
- LOST
- EVICT
- KILL

A presença equilibrada entre eventos de término e falha sugere comportamento dinâmico realista do sistema.

## Persistência e rastreabilidade
- Dataset validado salvo como: `trace_raw_validated.parquet`
- Summary salvo em: `01_ingest_validate_summary.json`
- Tipos ajustados para otimização de memória (category, float32, Int64)

## Conclusão metodológica
Este notebook estabelece:
- Integridade estrutural do dataset
- Consistência semântica das variáveis críticas
- Base confiável para normalização temporal (Notebook 02)